In [127]:
import cv2
from ultralytics import YOLO
import cvzone
import math

from sort import *

In [128]:
# creating web cam object
# cap = cv2.VideoCapture(0)
cap = cv2.VideoCapture('./Videos/cars.mp4')
#cap.set(3,1280) # for width
#cap.set(4,720) #for height

In [129]:
model = YOLO('./Yolo-Weights/yolov8l.pt')

In [130]:
# classes that will be detected and displayed
classNames = ["person", "bicycle", "car", "motorbike", "aeroplane", "bus", "train", "truck", "boat",
              "traffic light", "fire hydrant", "stop sign", "parking meter", "bench", "bird", "cat",
              "dog", "horse", "sheep", "cow", "elephant", "bear", "zebra", "giraffe", "backpack", "umbrella",
              "handbag", "tie", "suitcase", "frisbee", "skis", "snowboard", "sports ball", "kite", "baseball bat",
              "baseball glove", "skateboard", "surfboard", "tennis racket", "bottle", "wine glass", "cup",
              "fork", "knife", "spoon", "bowl", "banana", "apple", "sandwich", "orange", "broccoli",
              "carrot", "hot dog", "pizza", "donut", "cake", "chair", "sofa", "pottedplant", "bed",
              "diningtable", "toilet", "tvmonitor", "laptop", "mouse", "remote", "keyboard", "cell phone",
              "microwave", "oven", "toaster", "sink", "refrigerator", "book", "clock", "vase", "scissors",
              "teddy bear", "hair drier", "toothbrush" , "pen"
              ]

In [131]:
mask = cv2.imread('./mask.png')

In [132]:
# creating a instance for tracking
tracker = Sort(max_age=20,min_hits=3,iou_threshold=0.3)

# we will make a line and count after that 
limits = [400,297,673,297]
totalcounts=[]

In [133]:
while True:
    success, img = cap.read()
    #overlay mask on the image/webcam feed
    imageRegion = cv2.bitwise_and(img,mask)
    # results = model(img,stream=True)
    results = model(imageRegion,stream=True)
    
    detections = np.empty((0,5))
    
    for r in results:
        boxes = r.boxes
        for box in boxes:
            # xy1 are points and xy2 are height and width
            x1,y1,x2,y2 = box.xyxy[0]
            # converting them into integers instead for tensor 
            x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
            # cv2.rectangle(img,(x1,y1),(x2,y2),(255,0,255),3) #using cv2 reactangle
            w,h = x2-x1 , y2-y1
            bbox = int(x1),int(y1),int(w),int(h)
            
            # print(x1,y1,x2,y2)
            
            # cvzone.cornerRect(img,(x1,y1,w,h),l=9) #using cvzone rectange
            
            #confidence
            conf = math.ceil((box.conf[0]*100))/100 #for confidence values
            print(conf)
            #classnames
            cls = int(box.cls[0])
            #to display conf and object class on webcam
            currentclass = classNames[cls]
            
            if currentclass == "car" or currentclass == "truck" or currentclass == "bus" or currentclass == "motorbike" and conf>0.3 :
                # cvzone.putTextRect(img,f'{currentclass} {conf}',(max(0,x1),max(35,y1)) , scale=0.6,thickness=1,offset=3)
                # cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=5) #using cvzone rectange
                currentArrays = np.array([x1,y1,x2,y2,conf])
                detections = np.vstack((detections,currentArrays))
            
    resultsTracker = tracker.update(detections)
    
    cv2.line(img,(limits[0],limits[1]),(limits[2],limits[3]),(0,0,255),5)
    
    for result in resultsTracker:
        x1,y1,x2,y2,id = result
        # converting them into integers instead for tensor 
        x1,y1,x2,y2 = int(x1),int(y1),int(x2),int(y2)
        print(result)
        w,h = x2-x1 , y2-y1
        cvzone.cornerRect(img,(x1,y1,w,h),l=9 , rt=2,colorR=(255,0,0))
        # cvzone.putTextRect(img,f' {int(id)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
        
        cx , cy = x1+w//2,y1+h//2
        cv2.circle(img,(cx,cy),5,(255,0,255),cv2.FILLED)
        
        if limits[0] <cx< limits[2] and limits[1]-20<cy<limits[1]+20:
            # totalcounts.append(id)
            # cvzone.putTextRect(img,f' {int(totalcounts)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
            if totalcounts.count(id)==0:
                totalcounts.append(id)
                cvzone.putTextRect(img,f' {len(totalcounts)}',(max(0,x1),max(35,y1)) , scale=2,thickness=3,offset=10)
                
        
    cvzone.putTextRect(img,f'Count : {len(totalcounts)}',(90,90)) 
    
    cv2.imshow("Image",img)
    # cv2.imshow("ImageRegion",imageRegion)
    # cv2.waitKey(1)
    # Wait for a key event, but only for a short duration (10 milliseconds)
    # If the key is Esc (27), then break out of the loop and close the webcam
    if cv2.waitKey(1) == 27:
        break

# Release the webcam and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


0: 384x640 3 cars, 842.5ms
Speed: 14.8ms preprocess, 842.5ms inference, 21.4ms postprocess per image at shape (1, 3, 384, 640)



0.81
0.8
0.67
[        571         316         651         429         732]
[        378         348         495         457         731]
[        458         226         509         270         730]


0: 384x640 3 cars, 826.0ms
Speed: 8.0ms preprocess, 826.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



0.88
0.83
0.82
[     568.92      322.12      649.08      435.88         732]
[     373.98      352.02      493.02      462.98         731]
[     456.14      227.88      506.86      271.12         730]


0: 384x640 4 cars, 750.8ms
Speed: 0.0ms preprocess, 750.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



...

0.86
0.84
0.77
0.75
0.54
[     587.14      200.03      627.24      230.93         799]
[     500.72      200.34      572.77      245.24         798]
[     473.78      321.02      544.24      382.66         797]
[     413.53      250.53      487.98      337.71         793]
[     565.03      347.72      649.79      442.67         792]


0: 384x640 4 cars, 2 trucks, 705.8ms
Speed: 0.0ms preprocess, 705.8ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)



0.82
0.79
0.69
0.55
0.43
0.41
[      583.7      200.71      625.49      235.88         799]
[     495.17      200.55      569.27      249.42         798]
[     456.21         339      532.03      404.62         797]
[      397.9      256.03      477.47      351.01         793]
[     550.23      365.81         640      463.21         792]


0: 384x640 3 cars, 1 bus, 3 trucks, 700.0ms
Speed: 5.5ms preprocess, 700.0ms inference, 0.0ms postprocess per image at shape (1, 3, 384, 640)


0.81
0.77
0.71
0.59
0.42
0.42
0.35
[     581.68      201.77      623.83      240.74         799]
[      490.8      200.79      567.18      253.98         798]
[      435.2      360.28      517.35      431.43         797]
[     379.29      260.97      465.71       365.9         793]
[     535.75      385.23      633.66      474.65         792]
